In [1]:
# Cell 1: Import libraries and setup visualization
# Imports
import sys
import os
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Add project src to path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 100)

# Initialize random seed for reproducibility
np.random.seed(42)
random_seed_offset = 0


/Users/eri/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Cell 13: New Report Structure Demo - Realistic Oncologic Radiology Reports

print("🔬 NEW RADIOLOGY REPORT STRUCTURE DEMO")
print("=" * 60)

# Add src to path
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../src' if 'notebooks' in os.getcwd() else 'src'))

from synthrad.generator import generate_case, generate_report

# Generate sample cases for demonstration
print("📋 Generating sample cases for demonstration...")

# Case 1: Stage IV with multiple metastases
case1 = generate_case(
    seed=42,
    stage_dist={"I": 0.0, "II": 0.0, "III": 0.0, "IV": 1.0},
    lobe="RUL",
    patient_id="DEMO001",
    visit_number=1
)

# Case 2: Stage II with nodal disease
case2 = generate_case(
    seed=123,
    stage_dist={"I": 0.0, "II": 1.0, "III": 0.0, "IV": 0.0},
    lobe="LLL",
    patient_id="DEMO002", 
    visit_number=1
)

# Case 3: Stage I early disease
case3 = generate_case(
    seed=456,
    stage_dist={"I": 1.0, "II": 0.0, "III": 0.0, "IV": 0.0},
    lobe="LUL",
    patient_id="DEMO003",
    visit_number=1
)

print(f"✅ Generated 3 sample cases")
print(f"   Case 1: {case1.tnm.T}{case1.tnm.N}{case1.tnm.M} ({case1.tnm.stage_group}) - {case1.primary.lobe} {case1.primary.size_mm}mm")
print(f"   Case 2: {case2.tnm.T}{case2.tnm.N}{case2.tnm.M} ({case2.tnm.stage_group}) - {case2.primary.lobe} {case2.primary.size_mm}mm")
print(f"   Case 3: {case3.tnm.T}{case3.tnm.N}{case3.tnm.M} ({case3.tnm.stage_group}) - {case3.primary.lobe} {case3.primary.size_mm}mm")
print()

# Show CT CAP report for Case 1
print("📄 CT CHEST/ABDOMEN/PELVIS REPORT (Case 1 - Stage IV):")
print("-" * 60)
ct_report = generate_report(case1, modality="CT", include_recist=True)
print(ct_report)
print()

# Show PET-CT report for Case 2
print("📄 PET-CT REPORT (Case 2 - Stage II):")
print("-" * 60)
pet_report = generate_report(case2, modality="PET-CT", include_recist=True)
print(pet_report)
print()

# Show CT report without RECIST for Case 3
print("📄 CT REPORT WITHOUT RECIST (Case 3 - Stage I):")
print("-" * 60)
simple_report = generate_report(case3, modality="CT", include_recist=False)
print(simple_report)
print()


🔬 NEW RADIOLOGY REPORT STRUCTURE DEMO
📋 Generating sample cases for demonstration...
✅ Generated 3 sample cases
   Case 1: T1cN2M1c (IV) - RUL 23mm
   Case 2: T2aN0M0 (II) - LLL 36mm
   Case 3: T2aN0M0 (II) - LUL 45mm

📄 CT CHEST/ABDOMEN/PELVIS REPORT (Case 1 - Stage IV):
------------------------------------------------------------
TECHNIQUE: CT chest, abdomen, and pelvis with IV contrast. Axial images with multiplanar reconstructions.
COMPARISON: None.

FINDINGS:
Lungs/Primary: right upper lobe mass 23 mm, pleural abutment; invasion not excluded.
Mediastinum/Lymph nodes:
  12R node 14 mm short axis (pathologic).
  4L node 15 mm short axis (pathologic).
Pleura: No pleural effusion.
Abdomen/Pelvis:
  adrenal right nodule 11 mm, suspicious.
  peritoneum lesion 16 mm, suspicious for metastasis.
  Liver homogeneous in attenuation without focal mass.
  Adrenal glands without nodules.
Bones: No destructive osseous lesion.

RECIST:
Target lesions:
  1) right upper lobe mass: 23 mm; baseline m

In [3]:
# Cell 14: Lesion Writing Styles Comparison

print("🎨 LESION WRITING STYLES COMPARISON")
print("=" * 50)

# Add src to path
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../src' if 'notebooks' in os.getcwd() else 'src'))

from synthrad.generator import generate_case, generate_report
from synthrad.lexicons import RADIOLOGIST_STYLES, node_phrase, mm_desc, fmt_mm, compare_size

# Generate a sample case
case = generate_case(
    seed=789,
    stage_dist={"I": 0.0, "II": 0.0, "III": 0.0, "IV": 1.0},
    lobe="RLL",
    patient_id="STYLE_DEMO",
    visit_number=1
)

print(f"📋 Sample Case: {case.meta.patient_id}")
print(f"🫁 Primary: {case.primary.lobe} lobe, {case.primary.size_mm}mm")
print(f"🔗 Nodes: {len(case.nodes)} lymph nodes")
for node in case.nodes:
    print(f"   - {node.station}: {node.short_axis_mm}mm")
print(f"🌍 Metastases: {len(case.mets)} sites")
for met in case.mets:
    print(f"   - {met.site}: {met.size_mm}mm")
print()

# Show the same case in different writing styles
available_styles = list(RADIOLOGIST_STYLES.keys())

for i, style in enumerate(available_styles, 1):
    print(f"📝 STYLE {i}: {style.upper()}")
    print("-" * 40)
    
    # Create a copy of the case with the specific style
    style_case = case.model_copy()
    style_case.meta.radiologist_style = style
    
    # Generate report with this style
    report = generate_report(style_case, modality="CT", include_recist=True)
    print(report)
    print()

print("✅ STYLE COMPARISON COMPLETE")
print("=" * 50)

# Show utility functions
print("\n🛠️ UTILITY FUNCTIONS DEMO:")
print("-" * 30)

# Test mm_desc function
print("📏 Size descriptions:")
for size in [8, 12, 15, 25]:
    print(f"   {size}mm → {mm_desc(size)}")

# Test node_phrase function
print("\n🔗 Node phrase examples:")
for style in ["concise", "detailed", "academic"]:
    print(f"   {style}: {node_phrase('4R', 14, style=style)}")
    print(f"   {style}: {node_phrase('11L', 8, style=style)}")

# Test comparison function
print("\n📊 Size comparisons:")
print(f"   {compare_size(25, None)}")
print(f"   {compare_size(30, 25)}")
print(f"   {compare_size(20, 25)}")
print(f"   {compare_size(35, 25)}")

# Test fmt_mm function
print("\n📐 Measurement formatting:")
for size in [8, 15, 25, 120]:
    print(f"   {size}mm → {fmt_mm(size)}")


🎨 LESION WRITING STYLES COMPARISON
📋 Sample Case: STYLE_DEMO
🫁 Primary: RLL lobe, 49mm
🔗 Nodes: 0 lymph nodes
🌍 Metastases: 2 sites
   - peritoneum: 25mm
   - bone: 17mm

📝 STYLE 1: CONCISE
----------------------------------------
TECHNIQUE: CT chest, abdomen, and pelvis with IV contrast. Axial images with multiplanar reconstructions.
COMPARISON: None.

FINDINGS:
Lungs/Primary: right lower lobe pulmonary mass 49 mm with spiculated margins.
Mediastinum/Lymph nodes: No pathologic mediastinal adenopathy.
Pleura: No pleural effusion.
Abdomen/Pelvis:
  peritoneum lesion 25 mm, suspicious for metastasis.
  Liver homogeneous in attenuation without focal mass.
  Adrenal glands without nodules.
Bones: No destructive osseous lesion.

RECIST:
Target lesions:
  1) right lower lobe mass: 49 mm; baseline measurement.
  2) peritoneum nodule: 25 mm; baseline measurement.
  3) bone nodule: 17 mm; baseline measurement.
Non-target disease:
  - None.
New lesions:
  - None.
Overall RECIST: Baseline (no pri